# Azure AI Agent service - Function calling

<img src="https://learn.microsoft.com/en-us/azure/ai-services/agents/media/agent-service-the-glue.png" width=800>

> https://learn.microsoft.com/en-us/azure/ai-services/agents/

In [ ]:
#%pip install azure-ai-projects
#%pip install yfinance

In [1]:
import glob
import json
import ipyplot
import matplotlib.pyplot as plt
import os
import requests
import sys
import time
import yfinance as yf

#from azure.ai.projects import AIProjectClient
from azure.ai.agents import AgentsClient 
from azure.identity import DefaultAzureCredential
#from azure.ai.projects.models import FunctionTool, ToolSet, CodeInterpreterTool
from azure.ai.agents.models import (
    FunctionTool,
    ListSortOrder,
    RequiredFunctionToolCall,
    SubmitToolOutputsAction,
    ToolOutput,
)

from dotenv import load_dotenv

load_dotenv()
from user_functions import user_functions, azuremaps_weather

## 1. Agent definition - Weather forecasts

## Connect to AI Foundry Project

In [2]:
endpoint = os.getenv("PROJECT_ENDPOINT")
print(f"Using endpoint: {endpoint}")
credential = DefaultAzureCredential()

project_client = AgentsClient(endpoint=endpoint, credential=credential)

Using endpoint: https://aq-ai-foundry-sweden-central.services.ai.azure.com/api/projects/firstProject


### Testing the Azure Maps Weather function

We will use the Azure Maps service to provide weather information.

In [3]:
weather = azuremaps_weather("Toronto")
json.loads(weather)

{'weather_data': {'results': [{'dateTime': '2025-06-13T16:37:00-04:00',
    'phrase': 'Mostly sunny',
    'iconCode': 2,
    'hasPrecipitation': False,
    'isDayTime': True,
    'temperature': {'value': 16.1, 'unit': 'C', 'unitType': 17},
    'realFeelTemperature': {'value': 15.0, 'unit': 'C', 'unitType': 17},
    'realFeelTemperatureShade': {'value': 12.9, 'unit': 'C', 'unitType': 17},
    'relativeHumidity': 67,
    'dewPoint': {'value': 10.0, 'unit': 'C', 'unitType': 17},
    'wind': {'direction': {'degrees': 68.0, 'localizedDescription': 'ENE'},
     'speed': {'value': 25.9, 'unit': 'km/h', 'unitType': 7}},
    'windGust': {'speed': {'value': 25.9, 'unit': 'km/h', 'unitType': 7}},
    'uvIndex': 3,
    'uvIndexPhrase': 'Moderate',
    'visibility': {'value': 14.5, 'unit': 'km', 'unitType': 6},
    'obstructionsToVisibility': '',
    'cloudCover': 30,
    'ceiling': {'value': 12192.0, 'unit': 'm', 'unitType': 5},
    'pressure': {'value': 1021.3, 'unit': 'mb', 'unitType': 14},
    

### Agent

In [4]:
model = "gpt-4o-mini"
name = "gpt-4o-mini-weather-agent"
instructions = "You are a weather bot. Use the provided functions to help answer questions."

functions = FunctionTool(user_functions)

# Create an agent and run user's request with function calls
agent = project_client.create_agent(
    model=os.getenv("MODEL_DEPLOYMENT_NAME"),
    name=name,
    instructions=instructions,
    tools=functions.definitions,
)
print(f"Created agent, ID: {agent.id}")



Created agent, ID: asst_waUw0jSxyHOE3OODcnDbs8UJ


### Tests

In [5]:
prompt = "Hello, generate a full report for the weather in Toronto today"

# Create thread for communication
thread = project_client.threads.create()
print(f"Created thread ID = {thread.id}")

# Create message to thread
message = project_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=prompt,
)

print(f"Created message ID = {message.id}")

Created thread ID = thread_WYPMuKdq0aqF1s0CADPmZ7HB
Created message ID = msg_ueYRM5tJefGH9HjVqKalGmvJ


In [6]:
run = project_client.runs.create(thread_id=thread.id, agent_id=agent.id)
print(f"Created run, run ID: {run.id}")

Created run, run ID: run_rGvCMU8pNOt0iFZWTBezfRBl


In [7]:
    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = project_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolOutputsAction):
            tool_calls = run.required_action.submit_tool_outputs.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_outputs = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredFunctionToolCall):
                    try:
                        print(f"Executing tool call: {tool_call}")
                        output = functions.execute(tool_call)
                        tool_outputs.append(
                            ToolOutput(
                                tool_call_id=tool_call.id,
                                output=output,
                            )
                        )
                    except Exception as e:
                        print(f"Error executing tool_call {tool_call.id}: {e}")

            print(f"Tool outputs: {tool_outputs}")
            if tool_outputs:
                project_client.runs.submit_tool_outputs(thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs)

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")

Executing tool call: {'id': 'call_6XSSk0ZSk7EfGq4ipIBiIi8z', 'type': 'function', 'function': {'name': 'azuremaps_weather', 'arguments': '{"query":"Toronto"}'}}
Tool outputs: [{'tool_call_id': 'call_6XSSk0ZSk7EfGq4ipIBiIi8z', 'output': '{"weather_data": {"results": [{"dateTime": "2025-06-13T16:37:00-04:00", "phrase": "Mostly sunny", "iconCode": 2, "hasPrecipitation": false, "isDayTime": true, "temperature": {"value": 16.1, "unit": "C", "unitType": 17}, "realFeelTemperature": {"value": 15.0, "unit": "C", "unitType": 17}, "realFeelTemperatureShade": {"value": 12.9, "unit": "C", "unitType": 17}, "relativeHumidity": 67, "dewPoint": {"value": 10.0, "unit": "C", "unitType": 17}, "wind": {"direction": {"degrees": 68.0, "localizedDescription": "ENE"}, "speed": {"value": 25.9, "unit": "km/h", "unitType": 7}}, "windGust": {"speed": {"value": 25.9, "unit": "km/h", "unitType": 7}}, "uvIndex": 3, "uvIndexPhrase": "Moderate", "visibility": {"value": 14.5, "unit": "km", "unitType": 6}, "obstructionsTo

In [8]:
start   = run.started_at       
end     = run.completed_at
elapsed = end - start

iso_fmt = "%Y-%m-%d %H:%M:%S%z"
print(f"Start   : {start.strftime(iso_fmt)}")
print(f"End     : {end.strftime(iso_fmt)}")
print(f"Elapsed : {elapsed}  "
      f"({elapsed.total_seconds():.2f} seconds)")

run.usage

Start   : 2025-06-13 20:49:53+0000
End     : 2025-06-13 20:49:56+0000
Elapsed : 0:00:03  (3.00 seconds)


{'prompt_tokens': 1427, 'completion_tokens': 261, 'total_tokens': 1688, 'prompt_token_details': {'cached_tokens': 0}}

In [ ]:
# Fetch and log all messages
messages = project_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
for msg in messages:
    if msg.text_messages:
        last_text = msg.text_messages[-1]
        print(f"{msg.role}: {last_text.text.value}")

MessageRole.USER: Hello, generate a full report for the weather in Toronto today
MessageRole.AGENT: Here is the full weather report for Toronto today:

- Current Conditions: Mostly sunny
- Temperature: 16.1°C (feels like 15.0°C; in the shade feels like 12.9°C)
- Apparent Temperature: 18.3°C
- Wind: 25.9 km/h from the east-northeast (ENE), gusting to 25.9 km/h
- Humidity: 67%
- Dew Point: 10.0°C
- UV Index: 3 (Moderate)
- Visibility: 14.5 km
- Cloud Cover: 30%
- Pressure: 1021.3 mb and falling
- Ceiling: 12,192 meters
- Precipitation: None reported in the past 24 hours; 0.0 mm recorded
- Temperature Range (last 24 hours): Low of 12.8°C, high of 25.0°C

Summary: Toronto is experiencing mostly sunny weather today with comfortable temperatures and a moderate breeze. No precipitation has been recorded and visibility is excellent. The barometric pressure is falling slightly, suggesting potential changes ahead, but the day remains clear and pleasant.


### Other examples

In [10]:
prompt = "Hello, generate a full report for the weather in Mexico City today. Print the results in English, Spanish and French."

# Create thread for communication
thread = project_client.threads.create()
print(f"Created thread ID = {thread.id}")

# Create message to thread
message = project_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=prompt,
)

print(f"Created message ID = {message.id}")

Created thread ID = thread_7tLC0e9dd0jbV5oCvbmPvLOn
Created message ID = msg_vQHvbdr3FLt3uVqCZRrQArl9


In [11]:
run = project_client.runs.create(thread_id=thread.id, agent_id=agent.id)
print(f"Created run, run ID: {run.id}")

Created run, run ID: run_KYLAL6wqe4sahQqSm2Yu3svJ


In [12]:
    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = project_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolOutputsAction):
            tool_calls = run.required_action.submit_tool_outputs.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_outputs = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredFunctionToolCall):
                    try:
                        print(f"Executing tool call: {tool_call}")
                        output = functions.execute(tool_call)
                        tool_outputs.append(
                            ToolOutput(
                                tool_call_id=tool_call.id,
                                output=output,
                            )
                        )
                    except Exception as e:
                        print(f"Error executing tool_call {tool_call.id}: {e}")

            print(f"Tool outputs: {tool_outputs}")
            if tool_outputs:
                project_client.runs.submit_tool_outputs(thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs)

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")

Executing tool call: {'id': 'call_OwUTmfLoxnamFAiiDvSwMA5z', 'type': 'function', 'function': {'name': 'azuremaps_weather', 'arguments': '{"query":"Mexico City"}'}}
Tool outputs: [{'tool_call_id': 'call_OwUTmfLoxnamFAiiDvSwMA5z', 'output': '{"weather_data": {"results": [{"dateTime": "2025-06-13T14:35:00-06:00", "phrase": "Cloudy", "iconCode": 7, "hasPrecipitation": false, "isDayTime": true, "temperature": {"value": 22.2, "unit": "C", "unitType": 17}, "realFeelTemperature": {"value": 23.7, "unit": "C", "unitType": 17}, "realFeelTemperatureShade": {"value": 21.0, "unit": "C", "unitType": 17}, "relativeHumidity": 49, "dewPoint": {"value": 11.1, "unit": "C", "unitType": 17}, "wind": {"direction": {"degrees": 270.0, "localizedDescription": "W"}, "speed": {"value": 7.4, "unit": "km/h", "unitType": 7}}, "windGust": {"speed": {"value": 7.4, "unit": "km/h", "unitType": 7}}, "uvIndex": 3, "uvIndexPhrase": "Moderate", "visibility": {"value": 8.0, "unit": "km", "unitType": 6}, "obstructionsToVisibi

In [13]:
start   = run.started_at       
end     = run.completed_at
elapsed = end - start

iso_fmt = "%Y-%m-%d %H:%M:%S%z"
print(f"Start   : {start.strftime(iso_fmt)}")
print(f"End     : {end.strftime(iso_fmt)}")
print(f"Elapsed : {elapsed}  "
      f"({elapsed.total_seconds():.2f} seconds)")

run.usage

Start   : 2025-06-13 20:50:38+0000
End     : 2025-06-13 20:50:41+0000
Elapsed : 0:00:03  (3.00 seconds)


{'prompt_tokens': 1452, 'completion_tokens': 522, 'total_tokens': 1974, 'prompt_token_details': {'cached_tokens': 0}}

In [15]:
# Fetch and log all messages
messages = project_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
for msg in messages:
    if msg.text_messages:
        last_text = msg.text_messages[-1]
        print(f"{msg.role}: {last_text.text.value}")

MessageRole.USER: Hello, generate a full report for the weather in Mexico City today. Print the results in English, Spanish and French.
MessageRole.AGENT: Here is the full weather report for Mexico City today, presented in English, Spanish, and French:

---

**English:**  
Weather in Mexico City today:  
- Condition: Cloudy  
- Temperature: 22.2°C (feels like 23.7°C)  
- Humidity: 49%  
- Dew point: 11.1°C  
- Wind: 7.4 km/h, from the West  
- UV Index: 3 (Moderate)  
- Visibility: 8.0 km  
- Cloud cover: 95%  
- Pressure: 1024.4 mb (Falling)  
- Precipitation (past 24h): 23.4 mm  
- Min/Max temperature last 24h: 12.8°C / 22.8°C

---

**Español:**  
El tiempo en Ciudad de México hoy:  
- Condición: Nublado  
- Temperatura: 22.2°C (sensación de 23.7°C)  
- Humedad: 49%  
- Punto de rocío: 11.1°C  
- Viento: 7.4 km/h, desde el Oeste  
- Índice UV: 3 (Moderado)  
- Visibilidad: 8.0 km  
- Cobertura nubosa: 95%  
- Presión: 1024.4 mb (en descenso)  
- Precipitación (últimas 24h): 23.4 mm  

In [16]:
prompt = "Hello, print the UV informations for Oakville, Ontario today. Print the date as well of the weather data."

# Create thread for communication
thread = project_client.threads.create()
print(f"Created thread ID = {thread.id}")

# Create message to thread
message = project_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=prompt,
)

print(f"Created message ID = {message.id}")

Created thread ID = thread_vQCC9gP1DJSK2uVsjW1t2ksK
Created message ID = msg_3Vzh4u4XWRKR8KvxO6p4kvLv


In [17]:
run = project_client.runs.create(thread_id=thread.id, agent_id=agent.id)
print(f"Created run, run ID: {run.id}")

Created run, run ID: run_v5Oc5GtItqpyObGBPNxy37iR


In [18]:
    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = project_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolOutputsAction):
            tool_calls = run.required_action.submit_tool_outputs.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_outputs = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredFunctionToolCall):
                    try:
                        print(f"Executing tool call: {tool_call}")
                        output = functions.execute(tool_call)
                        tool_outputs.append(
                            ToolOutput(
                                tool_call_id=tool_call.id,
                                output=output,
                            )
                        )
                    except Exception as e:
                        print(f"Error executing tool_call {tool_call.id}: {e}")

            print(f"Tool outputs: {tool_outputs}")
            if tool_outputs:
                project_client.runs.submit_tool_outputs(thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs)

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")

Executing tool call: {'id': 'call_HIqJy7ZA338dm1HFLXc2jKa6', 'type': 'function', 'function': {'name': 'azuremaps_weather', 'arguments': '{"query":"Oakville, Ontario"}'}}
Tool outputs: [{'tool_call_id': 'call_HIqJy7ZA338dm1HFLXc2jKa6', 'output': '{"weather_data": {"results": [{"dateTime": "2025-06-13T16:42:00-04:00", "phrase": "Mostly sunny", "iconCode": 2, "hasPrecipitation": false, "isDayTime": true, "temperature": {"value": 16.0, "unit": "C", "unitType": 17}, "realFeelTemperature": {"value": 19.2, "unit": "C", "unitType": 17}, "realFeelTemperatureShade": {"value": 15.5, "unit": "C", "unitType": 17}, "relativeHumidity": 76, "dewPoint": {"value": 11.7, "unit": "C", "unitType": 17}, "wind": {"direction": {"degrees": 90.0, "localizedDescription": "E"}, "speed": {"value": 8.1, "unit": "km/h", "unitType": 7}}, "windGust": {"speed": {"value": 16.9, "unit": "km/h", "unitType": 7}}, "uvIndex": 3, "uvIndexPhrase": "Moderate", "visibility": {"value": 20.9, "unit": "km", "unitType": 6}, "obstruc

In [19]:
start   = run.started_at       
end     = run.completed_at
elapsed = end - start

iso_fmt = "%Y-%m-%d %H:%M:%S%z"
print(f"Start   : {start.strftime(iso_fmt)}")
print(f"End     : {end.strftime(iso_fmt)}")
print(f"Elapsed : {elapsed}  "
      f"({elapsed.total_seconds():.2f} seconds)")

run.usage

Start   : 2025-06-13 20:51:25+0000
End     : 2025-06-13 20:51:26+0000
Elapsed : 0:00:01  (1.00 seconds)


{'prompt_tokens': 1452, 'completion_tokens': 80, 'total_tokens': 1532, 'prompt_token_details': {'cached_tokens': 0}}

In [21]:
# Fetch and log all messages
messages = project_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
for msg in messages:
    if msg.text_messages:
        last_text = msg.text_messages[-1]
        print(f"{msg.role}: {last_text.text.value}")

MessageRole.USER: Hello, print the UV informations for Oakville, Ontario today. Print the date as well of the weather data.
MessageRole.AGENT: Here is the UV information for Oakville, Ontario today:

- Date of weather data: 2025-06-13
- UV Index: 3
- UV Index Level: Moderate

It is advisable to take basic sun protection measures if you plan to be outdoors for extended periods.


In [22]:
prompt = "Generate a weather report for Rome. This report is for a blog publication. Add emojis and adopt a fun style."

# Create thread for communication
thread = project_client.threads.create()
print(f"Created thread ID = {thread.id}")

# Create message to thread
message = project_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=prompt,
)

print(f"Created message ID = {message.id}")

Created thread ID = thread_uWFGmgCT0COAu6pbXp15p4o9
Created message ID = msg_RuUcT9gzqzkr9QlzpqVZMkZm


In [23]:
run = project_client.runs.create(thread_id=thread.id, agent_id=agent.id)
print(f"Created run, run ID: {run.id}")

Created run, run ID: run_B8umLQFi2evBFPT9wqQkUAIa


In [24]:
    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = project_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolOutputsAction):
            tool_calls = run.required_action.submit_tool_outputs.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_outputs = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredFunctionToolCall):
                    try:
                        print(f"Executing tool call: {tool_call}")
                        output = functions.execute(tool_call)
                        tool_outputs.append(
                            ToolOutput(
                                tool_call_id=tool_call.id,
                                output=output,
                            )
                        )
                    except Exception as e:
                        print(f"Error executing tool_call {tool_call.id}: {e}")

            print(f"Tool outputs: {tool_outputs}")
            if tool_outputs:
                project_client.runs.submit_tool_outputs(thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs)

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")

Executing tool call: {'id': 'call_aCDBHAach2pNAnGR0zhX5Wvy', 'type': 'function', 'function': {'name': 'azuremaps_weather', 'arguments': '{"query":"Rome"}'}}
Tool outputs: [{'tool_call_id': 'call_aCDBHAach2pNAnGR0zhX5Wvy', 'output': '{"weather_data": {"results": [{"dateTime": "2025-06-13T22:42:00+02:00", "phrase": "Mostly clear", "iconCode": 34, "hasPrecipitation": false, "isDayTime": false, "temperature": {"value": 27.9, "unit": "C", "unitType": 17}, "realFeelTemperature": {"value": 26.6, "unit": "C", "unitType": 17}, "realFeelTemperatureShade": {"value": 26.6, "unit": "C", "unitType": 17}, "relativeHumidity": 44, "dewPoint": {"value": 14.6, "unit": "C", "unitType": 17}, "wind": {"direction": {"degrees": 225.0, "localizedDescription": "SW"}, "speed": {"value": 9.6, "unit": "km/h", "unitType": 7}}, "windGust": {"speed": {"value": 22.6, "unit": "km/h", "unitType": 7}}, "uvIndex": 0, "uvIndexPhrase": "Low", "visibility": {"value": 24.1, "unit": "km", "unitType": 6}, "obstructionsToVisibil

In [25]:
start   = run.started_at       
end     = run.completed_at
elapsed = end - start

iso_fmt = "%Y-%m-%d %H:%M:%S%z"
print(f"Start   : {start.strftime(iso_fmt)}")
print(f"End     : {end.strftime(iso_fmt)}")
print(f"Elapsed : {elapsed}  "
      f"({elapsed.total_seconds():.2f} seconds)")

run.usage

Start   : 2025-06-13 20:52:07+0000
End     : 2025-06-13 20:52:10+0000
Elapsed : 0:00:03  (3.00 seconds)


{'prompt_tokens': 1448, 'completion_tokens': 259, 'total_tokens': 1707, 'prompt_token_details': {'cached_tokens': 0}}

In [26]:
# Fetch and log all messages
messages = project_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
for msg in messages:
    if msg.text_messages:
        last_text = msg.text_messages[-1]
        print(f"{msg.role}: {last_text.text.value}")

MessageRole.USER: Generate a weather report for Rome. This report is for a blog publication. Add emojis and adopt a fun style.
MessageRole.AGENT: 🌞 Roman Holiday Weather Report! 🏛️

Hello, sunshine lovers! Rome is truly rolling out the red carpet of summer vibes tonight. Here’s what’s happening in the Eternal City:

- Condition: Mostly clear skies 🌌 — The stars are out to play!
- Temperature: A warm and toasty 28°C (real feel: 27°C) 🍦
- Humidity: 44% — Just enough to keep your gelato from melting too fast
- Winds: A gentle southwest breeze at 10 km/h (kiss your hair, don’t mess it up) 🍃
- No rain in sight! Perfect for a moonlit stroll around the Colosseum 🌙
- UV Index: 0 (nighttime = safe for vampires 🧛)
- Visibility: Crystal clear at 24 km — say “ciao” to a stunning cityscape!

Rome is serving up la dolce vita weather tonight. Grab a terrazza table, watch the world go by, and enjoy that late-night espresso. The only thing hotter right now is your Italian amore! 🇮🇹💕

Stay tuned for mor

### Post processing

In [27]:
# List all agents in the project
print("Listing all agents in the project:")
agents = project_client.list_agents()
for agent in agents:
    print(f"Agent ID: {agent.id}, Name: {agent.name}, Model: {agent.model}, Instructions: {agent.instructions}")

Listing all agents in the project:
Agent ID: asst_waUw0jSxyHOE3OODcnDbs8UJ, Name: gpt-4o-mini-weather-agent, Model: gpt-4.1, Instructions: You are a weather bot. Use the provided functions to help answer questions.
Agent ID: asst_VDHKoVyINkNd65z3kLoXpUAS, Name: my-agent, Model: gpt-4.1, Instructions: You are helpful agent


In [29]:
# recurse through all agents and delete them adding a test to stop if the agent is not found
for agent in project_client.list_agents():
    try:
        print(f"Deleting agent ID: {agent.id}, Name: {agent.name}")
        project_client.delete_agent(agent.id)
    except Exception as e:
        print(f"Error deleting agent ID: {agent.id}, Name: {agent.name}, Error: {e}")

In [30]:
agents = project_client.list_agents()
for agent in agents:
    print(f"Agent ID: {agent.id}, Name: {agent.name}, Model: {agent.model}, Instructions: {agent.instructions}")